<center>
    <h2> <b> LINMA2450 : Combinatorial optimization </h2>
    <h1>   <b> Homework 3</h1> 
</center>
        
        
        
<center> 
    <h2> Adrien Banse (12501700) and Marine Branders (52691700)  </h2>
</center>

In [1]:
using Gurobi, JuMP
gurobi_env = Gurobi.Env()

Academic license - for non-commercial use only


Gurobi.Env(Ptr{Nothing} @0x00007f9ea2492c00)

# Naive formulation (Kantorovich Formulation)

In [4]:
W = 100
w = [25,35,45]
n = [10,20,30]
m = 3
l = 60

model_naive = Model(with_optimizer(Gurobi.Optimizer,gurobi_env))
set_optimizer_attribute(model_naive, "Presolve", 0)
set_optimizer_attribute(model_naive, "Heuristics", 0)
set_optimizer_attribute(model_naive, "Cuts", 0)
set_optimizer_attribute(model_naive, "TimeLimit", 120)

@variable(model_naive,y[1:l],Bin)
@variable(model_naive,x[1:m,1:l]>=0,Int)

@objective(model_naive,Min,sum(y))

@constraint(model_naive,sum(x,dims=2).>=n)
@constraint(model_naive,sum(transpose(x.*w),dims=2).<=W*y)

optimize!(model_naive)

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 63 rows, 240 columns and 420 nonzeros
Model fingerprint: 0xa5867c22
Variable types: 0 continuous, 240 integer (60 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 3e+01]
Variable types: 0 continuous, 240 integer (60 binary)

Root relaxation: objective 2.300000e+01, 134 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   23.00000    0   21          -   23.00000      -     -    0s
     0     0   23.00000    0   21          -   23.00000      -     -    0s
     0     2   23.00000    0   21          -   23.00000      -     -    0s
*  289   802             113      27.0000000   23.00000  14.8%   1.7    0s
* 1076  1464              76      26.0000000   23.00000  11.5%  

# Pattern based formulation (small problem)

In [112]:
W = 100
w = [25,35,45]
n = [10,20,30]
m = 3

model_pat_s = Model(with_optimizer(Gurobi.Optimizer,gurobi_env))
set_optimizer_attribute(model_pat_s, "Presolve", 0)
set_optimizer_attribute(model_pat_s, "Heuristics", 0)
set_optimizer_attribute(model_pat_s, "Cuts", 0)

a = [1 0 0 2 1 1 0 3 0 2 0 2 1 4;
     0 1 0 0 1 0 2 0 1 1 0 0 2 0;
     0 0 1 0 0 1 0 0 1 0 2 1 0 0]
    
l = size(a[1,:])[1]

@variable(model_pat_s,z[1:l]>=0,Int)
@objective(model_pat_s,Min,sum(z))
@constraint(model_pat_s,(a*z).>=n)
optimize!(model_pat_s)
value.(z)

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 3 rows, 14 columns and 20 nonzeros
Model fingerprint: 0xb4db470e
Variable types: 0 continuous, 14 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 3e+01]
Variable types: 0 continuous, 14 integer (0 binary)

Root relaxation: objective 2.500000e+01, 3 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0      25.0000000   25.00000  0.00%     -    0s

Explored 0 nodes (3 simplex iterations) in 0.02 seconds
Thread count was 8 (of 8 available processors)

Solution count 1: 25 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.500000000000e+01, best bound 2.500000000000e+01, gap 0.0000%


14-element Array{Float64,1}:
 -0.0
 -0.0
 -0.0
 -0.0
 -0.0
 -0.0
  0.0
 -0.0
 -0.0
 -0.0
 15.0
 -0.0
 10.0
 -0.0

# Pattern based formulation (large problem)

In [79]:
########
# data #
########

W = 5600
w = [1380,1520,1560,1710,1820,1880,1930,2000,2050,2100,2140,2150,2200]
n = [22,25,12,14,18,18,20,10,12,14,16,18,20]
m = 13

##########################
# build initial patterns #
##########################

using LinearAlgebra 
a = Matrix{Int}(I, m, m)

#################################
# find new interesting patterns #
#################################

eps = 1e-9
l=m
z_final=0
o_final=0
while true
    # define pattern problem with initial patterns
    model_pat_l = Model(with_optimizer(Gurobi.Optimizer,gurobi_env))
    set_optimizer_attribute(model_pat_l, "Presolve", 0)
    set_optimizer_attribute(model_pat_l, "Heuristics", 0)
    set_optimizer_attribute(model_pat_l, "Cuts", 0)
    set_optimizer_attribute(model_pat_l, "FeasibilityTol", eps)
    set_optimizer_attribute(model_pat_l, "OptimalityTol", eps)
    set_silent(model_pat_l)
    @variable(model_pat_l,z[1:l]>=0)
    @objective(model_pat_l,Min,sum(z))
    @constraint(model_pat_l,con_pat,(a*z).>=n)
    optimize!(model_pat_l)
        
    #define knapsack problem
    model_kp = Model(with_optimizer(Gurobi.Optimizer,gurobi_env))
    set_optimizer_attribute(model_kp, "Presolve", 0)
    set_optimizer_attribute(model_kp, "Heuristics", 0)
    set_optimizer_attribute(model_kp, "Cuts", 0)
    set_optimizer_attribute(model_kp, "FeasibilityTol", eps)
    set_optimizer_attribute(model_kp, "OptimalityTol", eps)
    set_silent(model_kp)
    @variable(model_kp,p[1:m]>=0,Int)
    lambda = zeros(m)
    for i = 1:m
        lambda[i] = dual(con_pat[i])
    end
    @objective(model_kp,Max,sum(lambda.*p))
    @constraint(model_kp,sum(p.*w)<=W)
    optimize!(model_kp)
    
    if objective_value(model_kp) - 1 <= eps
        z_final=value.(z)
        o_final=objective_value(model_pat_l)
        break
    else
        print(objective_value(model_kp))
        print(" : ")
        println(value.(p))
    end
    
    l=l+1
    a = [a value.(p)]
end

4.0 : [4.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0]
3.0 : [0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
3.0 : [-0.0, 3.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0]
3.0 : [-0.0, -0.0, -0.0, -0.0, 3.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0]
3.0 : [-0.0, -0.0, -0.0, 1.0, -0.0, 2.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0]
3.0 : [-0.0, -0.0, -0.0, 1.0, -0.0, -0.0, 2.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0]
2.3333333333333335 : [-0.0, 1.0, -0.0, -0.0, -0.0, -0.0, -0.0, 2.0, -0.0, -0.0, -0.0, -0.0, -0.0]
2.25 : [1.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 2.0, -0.0, -0.0, -0.0]
2.25 : [1.0, 0.0, 0.0, -0.0, 0.0, -0.0, 0.0, 0.0, 2.0, 0.0, -0.0, -0.0, -0.0]
2.0 : [-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 2.0, -0.0]
2.0 : [-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 2.0]
2.0 : [0.0, -0.0, -0.0, 0.0, -0.0, -0.0, -0.0, 0.0, 0.0, 0.0, 2.0, -0.0, -0.0]
1.3

In [81]:
println()
print("added : ")
println(l-m)

println()
print("solution (relaxed) : \n")
for z in z_final
    if z!=0.0
        print(z)
        print(" : ")
        println(a[:,findall(x->x==z, z_final)[1]])
    end
end
println()

print("objective value (relaxed) : ")
print(o_final)
println()

println()
print("solution (rounding up) : \n")
z_r = round.(z_final,RoundUp)
for z in z_r
    if z!=0.0
        print(z)
        print(" : ")
        println(a[:,findall(x->x==z, z_r)[1]])
    end
end
println()

print("objective value (rounding up) : ")
println(sum(z_r))


added : 30

solution (relaxed) : 
0.25000000000000044 : [4.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0]
7.0 : [1.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 2.0, -0.0, -0.0, -0.0]
1.3333333333333353 : [-0.0, 1.0, -0.0, -0.0, 1.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 1.0]
2.999999999999999 : [-0.0, -0.0, -0.0, 2.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 1.0, -0.0]
7.000000000000002 : [-0.0, -0.0, 1.0, -0.0, 0.0, 1.0, -0.0, -0.0, -0.0, -0.0, -0.0, 1.0, -0.0]
7.999999999999998 : [-0.0, 0.9999999999999999, -0.0, -0.0, -0.0, -0.0, 1.0, -0.0, -0.0, -0.0, -0.0, 1.0, -0.0]
8.000000000000002 : [-0.0, 0.0, -0.0, 1.0, 1.0, -0.0, -0.0, -0.0, 1.0, -0.0, -0.0, -0.0, -0.0]
3.666666666666666 : [0.0, -0.0, -0.0, -0.0, 1.0, 2.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0]
3.9999999999999982 : [1.0, 0.0, 0.0, 0.0, -0.0, -0.0, -0.0, 0.0, 1.0, -0.0, 1.0, -0.0, -0.0]
10.0 : [1.0, 0.0, 0.0, -0.0, 0.0, -0.0, -0.0, 1.0, -0.0, -0.0, -0.0, -0.0, 1.0]
4.999999999999998 